In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from numba import njit, float64, int64
from numba.types import Tuple
from multiprocessing import Pool, cpu_count

In [2]:
G = 6.67430e-11

In [3]:
def initialize_particles(N):
    masses = np.random.rand(N) * 1e20  # случайные массы
    positions = np.random.rand(N, 2) * 1e11  # случайные позиции
    velocities = np.random.rand(N, 2) * 1e3  # случайные скорости
    return masses, positions, velocities

In [4]:
def velocity_verlet(masses, positions, velocities, dt, steps):
    N = len(masses)
    traj = np.zeros((N, 2, steps))
    traj[:, :, 0] = positions
    pos = positions.copy()
    vel = velocities.copy()

    def compute_acc(pos):
        acc = np.zeros_like(pos)
        for i in range(N):
            for j in range(N):
                if i != j:
                    r = pos[j] - pos[i]
                    dist = np.linalg.norm(r) + 1e-10
                    acc[i] += G * masses[j] * r / dist**3
        return acc

    acc = compute_acc(pos)
    for t in range(1, steps):
        pos += vel * dt + 0.5 * acc * dt**2
        new_acc = compute_acc(pos)
        vel += 0.5 * (acc + new_acc) * dt
        acc = new_acc
        traj[:, :, t] = pos
    return traj

In [5]:
@njit(float64[:, :, :](float64[:], float64[:, :], float64[:, :], float64, int64))
def velocity_verlet_numba(masses, positions, velocities, dt, steps):
    N = masses.shape[0]
    traj = np.zeros((N, 2, steps), dtype=np.float64)
    traj[:, :, 0] = positions
    pos = positions.copy()
    vel = velocities.copy()
    acc = np.zeros((N, 2), dtype=np.float64)

    for i in range(N):
        for j in range(N):
            if i != j:
                r = positions[j] - positions[i]
                dist = np.sqrt(r[0]**2 + r[1]**2) + 1e-10
                acc[i, 0] += G * masses[j] * r[0] / dist**3
                acc[i, 1] += G * masses[j] * r[1] / dist**3

    for t in range(1, steps):
        pos += vel * dt + 0.5 * acc * dt**2
        new_acc = np.zeros((N, 2), dtype=np.float64)
        for i in range(N):
            for j in range(N):
                if i != j:
                    r = pos[j] - pos[i]
                    dist = np.sqrt(r[0]**2 + r[1]**2) + 1e-10
                    new_acc[i, 0] += G * masses[j] * r[0] / dist**3
                    new_acc[i, 1] += G * masses[j] * r[1] / dist**3
        vel += 0.5 * (acc + new_acc) * dt
        acc = new_acc
        traj[:, :, t] = pos

    return traj

In [6]:
def compute_acceleration_parallel(args):
    i, pos, masses = args
    N = len(masses)
    acc_i = np.zeros(2)
    for j in range(N):
        if i != j:
            r = pos[j] - pos[i]
            dist = np.linalg.norm(r) + 1e-10
            acc_i += G * masses[j] * r / dist**3
    return i, acc_i

def compute_acc_parallel(pos, masses, pool):
    N = len(masses)
    results = pool.map(compute_acceleration_parallel, [(i, pos, masses) for i in range(N)])
    acc = np.zeros_like(pos)
    for i, acc_i in results:
        acc[i] = acc_i
    return acc

def velocity_verlet_parallel(masses, positions, velocities, dt, steps, pool):
    N = len(masses)
    traj = np.zeros((N, 2, steps))
    traj[:, :, 0] = positions
    pos = positions.copy()
    vel = velocities.copy()

    acc = compute_acc_parallel(pos, masses, pool)
    for t in range(1, steps):
        pos += vel * dt + 0.5 * acc * dt**2
        new_acc = compute_acc_parallel(pos, masses, pool)
        vel += 0.5 * (acc + new_acc) * dt
        acc = new_acc
        traj[:, :, t] = pos
    return traj

In [7]:
def run_benchmark(N, steps=1000, dt=1e5):
    print('Current N = ' + str(N))
    masses, positions, velocities = initialize_particles(N)
    print('verle')
    t0 = time()
    velocity_verlet(masses, positions, velocities, dt, steps)
    sequential_time = time() - t0

    print('verle + numba')
    t0 = time()
    velocity_verlet_numba(masses, positions, velocities, dt, steps)
    numba_time = time() - t0
    
    print('verle + multi')
    t0 = time()
    #with Pool(processes=cpu_count()) as pool:
        #velocity_verlet_parallel(masses, positions, velocities, dt, steps, pool)
    #velocity_verlet_parallel(masses, positions, velocities, dt, steps)
    parallel_time = time() - t0

    return sequential_time, numba_time, parallel_time

In [ ]:
def plot_benchmarks():
    N_values = [100, 200, 400]
    sequential_times = []
    numba_times = []
    parallel_times = []

    for N in N_values:
        sequential_time1, numba_time1, parallel_time1 = run_benchmark(N)
        sequential_time2, numba_time2, parallel_time2 = run_benchmark(N)
        sequential_time3, numba_time3, parallel_time3 = run_benchmark(N)
        
        sequential_time = (sequential_time1 + sequential_time2 + sequential_time3) / 3
        numba_time = (numba_time1 + numba_time2 + numba_time3) / 3
        parallel_time = (parallel_time1 + parallel_time2 + parallel_time3) / 3
        
        sequential_times.append(sequential_time)
        numba_times.append(numba_time)
        parallel_times.append(parallel_time)

    plt.figure(figsize=(10, 5))
    plt.plot(N_values, sequential_times, label='Верле (последовательный)', marker='o')
    plt.plot(N_values, numba_times, label='Верле + Numba', marker='o')
    plt.plot(N_values, parallel_times, label='Верле + multiprocessing', marker='o')
    plt.title('Время работы методов Верле при разных N')
    plt.xlabel('Количество частиц (N)')
    plt.ylabel('Время работы (с)')
    plt.legend()
    plt.grid()
    plt.show()
    
    return sequential_time, numba_time, parallel_time

In [9]:
def plot_speedup(sequential_time, numba_time, parallel_time):
    N_values = [100, 200, 400]
    speedup_numba = []
    speedup_parallel = []

    for N in N_values:
        speedup_numba.append(sequential_time / numba_time)
        #speedup_parallel.append(sequential_time / parallel_time)

    plt.figure(figsize=(10, 5))
    plt.plot(N_values, speedup_numba, label='Ускорение (Numba)', marker='o')
    #plt.plot(N_values, speedup_parallel, label='Ускорение (multiprocessing)', marker='o')
    plt.title('Ускорение методов Верле по сравнению с последовательным методом')
    plt.xlabel('Количество частиц (N)')
    plt.ylabel('Ускорение')
    plt.legend()
    plt.grid()
    plt.show()

In [10]:
def main():
    sequential_time, numba_time, parallel_time = plot_benchmarks()
    plot_speedup(sequential_time, numba_time, parallel_time)


if __name__ == "__main__":
    main()

Current N = 100
verle
verle + numba
verle + multi
Current N = 100
verle
verle + numba
verle + multi
Current N = 100
verle
verle + numba
verle + multi


UnboundLocalError: local variable 'numba_time' referenced before assignment